In [ ]:
############################################################################################################################
### Projeto da Disciplina de PO-233                                                                                        #
### Título: Identificação de dados biométricos utilizando redes Wireless e aprendizado de máquina                          #
### Alunos:                                                                                                                #
###         Ágney Lopes Roth Ferraz                                                                                        #
###	        Alexandre Bellargus Silva da Costa                                                                             #
###	        Carlos Renato de Andrade Figueiredo                                                                            #
###	        Gioliano de Oliveira Braga                                                                                     #
###	        Paulo Ricardo Sousa Fonteles de Castro                                                                         #
###	        Wagner Comin Sonaglio                                                                                          #
############################################################################################################################

from time import time, strftime, gmtime
inicio = time()
print("Iniciando algoritmo...\n")

## -------------------- 01. Importação de Bibliotecas -------------------- 

Esta etapa organiza e importa todas as bibliotecas utilizadas ao longo do projeto. O código está estruturado em seções temáticas com comentários que facilitam a navegação e manutenção. Os grupos incluem:

- **Sistema de Arquivos**: Operações com diretórios e caminhos (`os`, `shutil`, `pathlib`)
- **Manipulação de Dados**: Leitura, organização e processamento vetorial/tabular com `numpy` e `pandas`
- **Visualização**: Geração de gráficos com `matplotlib`, `seaborn` e visualização de árvores com `graphviz`
- **Temporização**: Medição de tempo de execução com `timeit`
- **Modelos de ML**: Diversos algoritmos supervisionados, como árvores, SVM, kNN, ensembles e redes neurais
- **Utilidades de ML**: Pré-processamento, normalização, PCA e clonagem de modelos
- **Validação**: Estratégias como hold-out, k-fold, Leave-One-Out e busca de hiperparâmetros (`GridSearchCV`)
- **Métricas**: Avaliação de desempenho por acurácia, precisão, recall, F1-score, matriz de confusão e curva ROC
- **Dados Sintéticos**: Geração de datasets artificiais para testes visuais
- **Warnings**: Tratamento de alertas comuns como conversão e convergência
- **Serialização**: Salvamento e carregamento de modelos com `joblib`

Este bloco é essencial para garantir que todas as ferramentas necessárias estejam disponíveis antes da execução do pipeline de aprendizado de máquina.

In [ ]:
# ===============================================================
# Bibliotecas do Sistema e Manipulação de Arquivos
# ===============================================================
import os                            # Operações com sistema de arquivos
import shutil                        # Operações para apagar arquivos e diretórios
from pathlib import Path             # Caminhos multiplataforma

# ===============================================================
# Manipulação de Dados
# ===============================================================
import numpy as np                  # Operações numéricas e vetoriais
import pandas as pd                 # Leitura e manipulação de dados tabulares
import re                           # Necessário para limpar nomes de arquivos

# ===============================================================
# Visualização de Dados
# ===============================================================
import matplotlib.pyplot as plt      # Gráficos gerais
import seaborn as sns                # Visualizações estatísticas
from graphviz import Source          # Visualização de árvores (formato .dot)
from sklearn.inspection import DecisionBoundaryDisplay  # Utilizado para visualizar as fronteiras de decisão dos classificadores em gráficos 2D

# ===============================================================
# Temporização
# ===============================================================
from timeit import default_timer as timer  # Cronômetro para medir tempo de execução

# ===============================================================
# Aprendizado de Máquina - Modelos
# ===============================================================
from sklearn.tree import DecisionTreeClassifier, export_graphviz    # Árvores de decisão
from sklearn.neighbors import KNeighborsClassifier                   # kNN
from sklearn.naive_bayes import GaussianNB                           # Naive Bayes
from sklearn.svm import SVC, LinearSVC                               # Máquinas de Vetores de Suporte
from sklearn.linear_model import SGDClassifier                       # Gradiente Estocástico
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier  # Ensemble
from sklearn.neural_network import MLPClassifier                     # Redes Neurais
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis  # Análise discriminante

# ===============================================================
# Aprendizado de Máquina - Utilidades
# ===============================================================
from sklearn.base import clone                                       # Clonagem de modelos
from sklearn.preprocessing import StandardScaler                     # Normalização de atributos
from sklearn.decomposition import PCA                                # Redução de dimensionalidade

# ===============================================================
# Validação e Seleção de Modelos
# ===============================================================
from sklearn.model_selection import (
    train_test_split,             # Separação treino/validação
    cross_val_score,              # Validação cruzada simples
    StratifiedKFold,              # Validação estratificada
    GridSearchCV,                 # Busca em grade de hiperparâmetros
    LeaveOneOut                   # Validação Leave-One-Out
)

# ===============================================================
# Métricas de Avaliação
# ===============================================================
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,            # Métricas principais
    confusion_matrix, classification_report,                            # Relatórios e matrizes
    roc_curve, auc                                                      # Curvas ROC e área sob a curva
)

# ===============================================================
# Geração de Dados Sintéticos
# ===============================================================
from sklearn.datasets import make_moons, make_circles, make_classification  # Bases de teste

# ===============================================================
# Tratamento de Warnings
# ===============================================================
import warnings                        # Controle de avisos
from sklearn.exceptions import (
    DataConversionWarning,             # Conversão de tipos
    ConvergenceWarning                 # Convergência de modelos (MLP, etc.)
)

# ===============================================================
# Serialização de Modelos
# ===============================================================
import joblib                          # Salvamento e carregamento de modelos

## -------------------- 02. Leitura dos Arquivos Disponíveis -------------------- 

Esta etapa realiza a leitura da pasta `/db/` em busca de arquivos `.csv`, que contêm os dados capturados do Channel State Information (CSI). Esses arquivos são listados e exibidos no terminal, para que o usuário escolha qual utilizar nas etapas seguintes de pré-processamento e modelagem.


In [ ]:
# Define o caminho da pasta onde estão os arquivos .csv
db_path = Path('./db')

# Lista todos os arquivos com extensão .csv no diretório /db/
arquivos_db = list(db_path.glob('*.csv'))

# Imprime uma mensagem informativa no terminal
print("Arquivos csv disponíveis na pasta /db/:")

# Percorre cada arquivo encontrado na lista
for arquivo in arquivos_db:
    # Imprime apenas o nome do arquivo (sem o caminho completo)
    print("-", arquivo.name)

## -------------------- 03. Definição dos Arquivos de Dados e Carregamento Inicial -------------------- 

Nesta etapa são definidos os caminhos para os arquivos `.csv` que representam diferentes cenários do experimento.

In [ ]:
# Define o diretório base onde estão localizados os arquivos .csv
db_dir = Path('db')

# Define o caminho completo do arquivo principal de treino e validação
arquivo_treino_validacao = db_dir / 'db_3200.csv'

# Define o caminho do arquivo usado para teste final com dados nunca vistos
arquivo_teste_real = db_dir / 'db_800.csv'

# Define o caminho do arquivo com 271 portadoras
arquivo_pca_271_portadoras = db_dir / 'db_400.csv'

# Carrega os dados do arquivo principal em um DataFrame usando pandas
df = pd.read_csv(arquivo_treino_validacao)

# Exibe as primeiras linhas do DataFrame para verificação inicial
df.head()

## -------------------- 04. Preparação do Diretório de Resultados --------------------

Esta etapa garante que o diretório `results/`, onde serão salvos todos os gráficos, matrizes e relatórios do experimento, esteja limpo e pronto para uso. 
A pasta é criada caso não exista e todos os arquivos e subpastas antigos dentro dela são removidos. Isso evita acúmulo de resultados de execuções anteriores e garante que apenas os dados atualizados estejam disponíveis ao final da execução dos experimentos.


In [ ]:
# Define o caminho da pasta onde os resultados serão salvos
results_path = Path("results")

# Cria a pasta 'results' e todas as subpastas necessárias, caso ainda não existam
results_path.mkdir(parents=True, exist_ok=True)

# Percorre todos os itens (arquivos e subpastas) dentro da pasta 'results'
for item in results_path.iterdir():
    
    # Se o item for um arquivo, remove com unlink()
    if item.is_file():
        item.unlink()
    
    # Se o item for uma subpasta, remove com rmtree()
    elif item.is_dir():
        shutil.rmtree(item)

# Informa no terminal que a pasta está pronta para uso
print("Diretório 'results' limpo e pronto para uso.")

## -------------------- 05. Pré-processamento: Conversão de Gênero para Valores Numéricos --------------------

Antes de aplicar algoritmos de aprendizado de máquina, é necessário que os dados estejam em um formato numérico. Esta função realiza a conversão da coluna `gender` — originalmente contendo valores categóricos `'f'` (female) e `'m'` (male) — para valores numéricos `0` e `1`, respectivamente.
Essa conversão facilita o treinamento dos modelos, que esperam entradas numéricas como rótulos para classificação. A função modifica a coluna diretamente no DataFrame e retorna o novo conjunto tratado.

In [ ]:
# Define uma função para converter a coluna 'gender' de categórico para numérico
def convert_gender_to_numeric(df):
    
    # Substitui os valores 'f' por 0 e 'm' por 1 na coluna 'gender'
    df['gender'] = df['gender'].replace({'f': 0, 'm': 1})
    
    # Retorna o DataFrame com a coluna 'gender' convertida
    return df

## -------------------- 06. PCA com Visualização Bidimensional (8 e 271 subportadoras) --------------------

Esta etapa realiza uma **análise exploratória visual** por meio da técnica de **PCA (Principal Component Analysis)**. O objetivo é observar se existe separação visual entre homens e mulheres com base nas subportadoras do CSI.

Duas versões do conjunto de dados são analisadas:
- **271 subportadoras**: base `db_400.csv`, com dimensionalidade reduzida via PCA após pré-processamento.
- **8 subportadoras**: base original `db_3200.csv`, com menos atributos por amostra.

Para ambas, aplica-se:
- Normalização com média zero e variância um (usando `StandardScaler`)
- Redução para 2 componentes principais (`PCA(n_components=2)`)
- Geração de gráficos de dispersão com `Seaborn`, destacando as classes por cor.

As imagens são salvas nos arquivos:
- `results/pca_271p.png`
- `results/pca_8p.png`

In [ ]:
# Carrega os dados da base com 271 subportadoras (após PCA externo ou seleção)
df = pd.read_csv(arquivo_pca_271_portadoras)
# Separa a variável alvo: 0 para mulher, 1 para homem
y = df.iloc[:, 0]
# Separa as colunas de atributos (subportadoras)
X = df.iloc[:, 1:]
# Cria o objeto de normalização (média zero, variância 1)
scaler = StandardScaler()
# Aplica normalização aos atributos
X_scaled = scaler.fit_transform(X)
# Converte os rótulos numéricos para texto (para legenda do gráfico)
labels = y.map({0: 'Men', 1: 'Women'})
# Cria um objeto PCA para reduzir para 2 componentes principais
pca = PCA(n_components=2)
# Aplica o PCA sobre os dados normalizados
X_pca = pca.fit_transform(X_scaled)
# Cria uma nova figura com tamanho 8x6
plt.figure(figsize=(8, 6))
# Gera gráfico de dispersão com cores diferentes para cada gênero
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')
# Define o título do gráfico
plt.title('PCA Dispersion by Gender - 271 subcarriers')
# Rótulo do eixo X
plt.xlabel("Principal Component 1")
# Rótulo do eixo Y
plt.ylabel("Principal Component 2")
# Exibe legenda
plt.legend()
# Exibe grade no fundo do gráfico
plt.grid(True)
# Ajusta layout para evitar cortes nos elementos do gráfico
plt.tight_layout()
# Salva a imagem gerada no diretório de resultados
plt.savefig("results/pca_271p.png", dpi=300)
# Exibe caminho salvo no terminal
print("PCA 271p: results/pca_271p.png")
# Fecha a figura (boa prática para liberar memória)
plt.close()

# Carrega os dados da base com 8 subportadoras (original, sem redução externa)
df = pd.read_csv(arquivo_treino_validacao)
# Separa a variável alvo: 0 para mulher, 1 para homem
y = df.iloc[:, 0]
# Separa os atributos (subportadoras)
X = df.iloc[:, 1:]
# Cria o objeto de normalização
scaler = StandardScaler()
# Aplica a normalização
X_scaled = scaler.fit_transform(X)
# Converte rótulos numéricos para texto para uso no gráfico
labels = y.map({0: 'Men', 1: 'Women'})
# Cria o objeto PCA com 2 componentes
pca = PCA(n_components=2)
# Reduz os dados para 2D com PCA
X_pca = pca.fit_transform(X_scaled)
# Cria uma nova figura com tamanho 8x6
plt.figure(figsize=(8, 6))
# Gera gráfico de dispersão colorido por classe
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],
                hue=y.map({0: 'Women', 1: 'Men'}), palette='Set1')
# Título do gráfico
plt.title('PCA Dispersion by Gender - 8 subcarriers')
# Rótulo do eixo X
plt.xlabel("Principal Component 1")
# Rótulo do eixo Y
plt.ylabel("Principal Component 2")
# Exibe legenda
plt.legend()
# Ativa a grade
plt.grid(True)
# Ajusta layout para evitar cortes
plt.tight_layout()
# Salva o gráfico no diretório de resultados
plt.savefig("results/pca_8p.png", dpi=300)
# Exibe caminho salvo no terminal
print("PCA 8p: results/pca_8p.png")
# Fecha a figura para liberar memória
plt.close()

## -------------------- 07. Visualização da Dispersão entre Portadoras (Pairplot) -------------------- 

Nesta etapa, é gerado um gráfico do tipo **pairplot**, que permite visualizar as relações entre diferentes subportadoras do CSI, separadas por gênero. O gráfico mostra como as distribuições e correlações entre os sinais das portadoras podem indicar diferenças entre homens e mulheres.

A base utilizada é o de treino e validação, com foco nas seguintes 8 portadoras:
- `rpi1_sc-118`, `rpi1_sc-111`, `rpi1_sc2`, `rpi1_sc32`
- `rpi1_sc67`, `rpi1_sc106`, `rpi1_sc120`, `rpi1_sc121`

O gráfico gerado é salvo no arquivo `results/pair_plot.png` e será usado para análise visual de separabilidade entre as classes antes do treinamento dos modelos.

In [ ]:
# Lê o banco de dados de treino/teste
df = pd.read_csv(arquivo_treino_validacao)

# Cria o diretório 'results' caso ainda não exista
Path("results").mkdir(exist_ok=True)

# Define os nomes das 8 subportadoras a serem usadas no gráfico
portadoras = ['rpi1_sc-118', 'rpi1_sc-111', 'rpi1_sc2', 'rpi1_sc32',
              'rpi1_sc67', 'rpi1_sc106', 'rpi1_sc120', 'rpi1_sc121']

# Cria um novo DataFrame contendo apenas as subportadoras selecionadas e a coluna de gênero
df_plot = df[portadoras + ['gender']].copy()

# Converte os rótulos 0 e 1 para 'Women' e 'Men' para fins de visualização
df_plot['gender'] = df_plot['gender'].map({0: 'Women', 1: 'Men'})

# Cria um gráfico de dispersão múltipla entre todas as subportadoras
plot = sns.pairplot(df_plot, hue='gender', palette='Set1', corner=True)

# Define o título do gráfico acima da figura
plt.suptitle("Carrier Dispersion by Gender", y=1.02)

# Salva o gráfico em alta resolução no diretório de resultados
plot.savefig("results/pair_plot.png", dpi=300)

# Fecha a figura para liberar memória
plt.close()

# Exibe uma mensagem indicando o caminho do arquivo salvo
print("Pairplot salvo em: results/pair_plot.png")

## -------------------- 08. Função de Geração do Boxplot Melhorado --------------------

Esta função é utilizada para **visualizar graficamente o desempenho dos algoritmos** de aprendizado de máquina usando boxplots. Ela recebe um dicionário contendo listas de scores (ex: acurácias) de diferentes classificadores e gera um gráfico comparativo.

Funcionalidades incluídas:
- Conversão de entradas não-string para o eixo Y, se necessário.
- Remoção de valores `NaN` das listas para evitar falhas no plot.
- Salvamento do gráfico em alta definição no local especificado.

Este tipo de gráfico ajuda a identificar visualmente a **dispersão dos resultados** e a **robustez** de cada modelo.

In [ ]:
# Define uma função para gerar e salvar um gráfico boxplot com os resultados dos modelos
def plot_boxplot(result: dict, alg_name, boxplot_file, y_label_text="Score", titulo="Boxplot de Resultados"):
    
    # Converte o dicionário de resultados em DataFrame (não é usado diretamente no plot, mas pode ser útil)
    df_result = pd.DataFrame.from_dict(result)

    # Verifica se o rótulo do eixo Y é uma string; se não for, converte
    if not isinstance(y_label_text, str):
        print(f"Warning: y_label_text era {type(y_label_text)}. Convertendo.")
        y_label_text = str(y_label_text)

    # Define o estilo padrão do matplotlib
    plt.style.use('default')

    # Define o tamanho da figura do boxplot
    plt.figure(figsize=(10, 6))

    # Remove valores NaN das listas de resultados para evitar erro no gráfico
    cleaned = [pd.Series(v).dropna() for v in result.values()]

    # Gera o gráfico boxplot usando os dados limpos
    plt.boxplot(cleaned, tick_labels=list(result.keys()))

    # Define o título do gráfico
    plt.title(titulo)

    # Define o rótulo do eixo Y
    plt.ylabel(y_label_text)

    # Rotaciona os rótulos do eixo X para melhor visualização
    plt.xticks(rotation=15)

    # Adiciona grade horizontal ao gráfico
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)

    # Ajusta automaticamente os elementos do layout
    plt.tight_layout()

    # Salva o gráfico no arquivo especificado com alta resolução
    plt.savefig(boxplot_file, dpi=300)

    # Fecha o gráfico para liberar memória
    plt.close()

    # Exibe mensagem informando onde o gráfico foi salvo
    print(f"Boxplot salvo em: {boxplot_file}")

## -------------------- 09. Avaliação com Hold-Out (com 20% de validação) --------------------

Este bloco realiza a **validação hold-out**, separando 80% dos dados para treino e 20% para validação. Ele executa os seguintes passos:

- Criação e limpeza do diretório `results/hold-out`.
- Divisão dos dados em treino e validação com estratificação para manter a proporção de classes.
- Treinamento de seis modelos diferentes:
  - kNN (3 vizinhos)
  - Árvore de Decisão (profundidade 5)
  - Árvore Grande (sem limitação)
  - Naive Bayes
  - SVM Linear (C=0.025)
  - SVM com kernel RBF (γ=2, C=1)

Para cada modelo:
- Realiza o treino e predição.
- Calcula acurácia e gera relatório com precisão, recall e F1-score.
- Salva a matriz de confusão como imagem.
- Mantém registro da melhor acurácia encontrada.

Ao final:
- Gera um boxplot comparando as acurácias dos modelos.
- Gera um barplot com os valores individuais.
- Salva o nome do melhor modelo e a acurácia final em `.txt`.
- Serializa o modelo vencedor com `joblib` para uso futuro.

Todos os resultados são armazenados na pasta `results/hold-out/`.

In [ ]:
# Lê o banco de dados de treino/teste
df = pd.read_csv(arquivo_treino_validacao)

# Garante que o diretório de resultados para Hold-Out exista
output_dir = Path("results/hold-out")
output_dir.mkdir(parents=True, exist_ok=True)

# Limpa todos os arquivos e pastas dentro do diretório de resultados
for item in output_dir.iterdir():
    if item.is_file():
        item.unlink()
    elif item.is_dir():
        shutil.rmtree(item)

# Define o caminho do arquivo de relatório
relatorio_path = Path("results/hold-out/relatorio.txt")

# Separa os atributos (X) e os rótulos (y)
X = df.drop('gender', axis=1)
y = df['gender']

# Divide o conjunto em treino (80%) e validação (20%) com estratificação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Exibe as dimensões dos conjuntos gerados
print(f"Treino: {X_train.shape}, Validação: {X_val.shape}")

# Define os modelos que serão testados
modelos = {
    "kNN": KNeighborsClassifier(3),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Big Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "SVM Linear": SVC(kernel="linear", C=0.025, probability=True),
    "SVM RBF": SVC(gamma=2, C=1, probability=True)
}

# Inicializa os dicionários para armazenar os resultados
modelos_treinados = {}
acuracias = {}
resultados_val = {}
relatorio_final = ""
melhor_acc = 0
melhor_nome = ""
melhor_modelo = None

# Abre o arquivo de relatório para escrita
with open(relatorio_path, "w", encoding="utf-8") as f:
    
    # Escreve o cabeçalho do relatório
    f.write("Relatório Hold-Out (20% de validação):\n\n")
    
    # Itera sobre os modelos definidos
    for nome, modelo in modelos.items():
        
        # Clona o modelo para garantir independência entre execuções
        clf = clone(modelo)
        
        # Treina o modelo com os dados de treino
        clf.fit(X_train, y_train)
        
        # Salva o modelo treinado
        modelos_treinados[nome] = clf        

        # Realiza a predição sobre os dados de validação
        y_val_pred = clf.predict(X_val)
        
        # Calcula a acurácia do modelo
        acc = accuracy_score(y_val, y_val_pred)
        
        # Armazena a acurácia
        acuracias[nome] = acc
        resultados_val[nome] = [acc]

        # Gera o relatório detalhado
        relatorio = classification_report(
            y_val, y_val_pred, target_names=["Women", "Men"], zero_division=0, digits=4
        )
        
        # Escreve os resultados no relatório em arquivo
        f.write(f"Modelo: {nome}\n")
        f.write(relatorio)
        f.write("\n" + "-" * 60 + "\n")
        
        # Também imprime no terminal para feedback visual
        print(f"\nModelo: {nome}")
        print(relatorio)
        print("-" * 60)

        # Gera a matriz de confusão para o modelo atual
        cm = confusion_matrix(y_val, y_val_pred, labels=[0, 1])
        
        # Cria figura para o heatmap
        plt.figure(figsize=(6, 5))
        
        # Cria gráfico da matriz de confusão
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["Women", "Men"], yticklabels=["Women", "Men"])
        
        # Define os rótulos e título do gráfico
        plt.xlabel("Classe Predita")
        plt.ylabel("Classe Real")
        plt.title(f"Matriz de Confusão - {nome}")
        
        # Ajusta layout
        plt.tight_layout()
        
        # Salva a imagem gerada
        plt.savefig(f"results/hold-out/{nome}_matriz_confusao.png", dpi=300)
        plt.close()

        # Atualiza o melhor modelo, se aplicável
        if acc > melhor_acc:
            melhor_acc = acc
            melhor_nome = nome
            melhor_modelo = clf

# Gera gráfico boxplot com as acurácias
plot_boxplot(
    result=resultados_val,
    alg_name="Todos",
    boxplot_file="results/hold-out/boxplot.png",
    y_label_text="Acurácia",
    titulo="Boxplot - Hold-Out (20% Validação)"
)

# Cria DataFrame com acurácias para barplot
df_plot = pd.DataFrame({
    "Modelo": list(acuracias.keys()),
    "Acurácia": list(acuracias.values())
})

# Cria figura do barplot
plt.figure(figsize=(10, 6))
sns.barplot(x="Modelo", y="Acurácia", data=df_plot)

# Define limites e títulos do gráfico
plt.ylim(0, 1.05)
plt.title("Acurácia dos Modelos - Hold-Out (20%)")
plt.ylabel("Acurácia")
plt.xlabel("Modelos")

# Adiciona grade horizontal
plt.grid(True, axis="y", linestyle="--", alpha=0.6)
plt.xticks(rotation=15)

# Ajusta o layout
plt.tight_layout()

# Salva o gráfico
plt.savefig("results/hold-out/barplot.png", dpi=300)
plt.close()

# Exibe mensagem no terminal
print("Barplot salvo em: results/hold-out/barplot.png")

# Salva o nome e acurácia do melhor modelo em .txt
with open("results/hold-out/melhor_modelo.txt", "w", encoding="utf-8") as f:
    f.write(f"Melhor modelo: {melhor_nome}\nAcurácia: {melhor_acc:.4f}")

# Salva o objeto do melhor modelo usando joblib
joblib.dump(melhor_modelo, "results/hold-out/melhor_modelo.pkl")

# Mensagens finais de conclusão
print("\nAnálise concluída com sucesso!")
print(f"\nMelhor modelo: {melhor_nome} (Acurácia: {melhor_acc:.4f})")
print("Resultados salvos em: results/hold-out/")

## -------------------- 10. Validação com Leave-One-Out (LOO) --------------------

Este bloco realiza uma avaliação completa com a técnica **Leave-One-Out (LOO)**, onde cada amostra do conjunto é usada uma vez como teste e o restante como treino.

**Passos executados:**

1. Carrega o dataset de treino/validação.
2. Limpa o diretório `results/leave-one-out`.
3. Define seis classificadores:
   - Árvores (Decision Tree com e sem limitação de profundidade - Big Tree)
   - kNN
   - Naive Bayes
   - SVM Linear
   - SVM RBF (via `SVC`)
4. Para cada modelo:
   - Executa o LOO.
   - Agrupa acurácias em blocos de 100.
   - Gera relatório e matriz de confusão.
   - Atualiza o melhor modelo com base na acurácia média.
5. Ao final:
   - Gera gráficos (`boxplot.png`, `barplot.png`).
   - Treina o melhor modelo com todos os dados.
   - Salva o modelo (`melhor_modelo.pkl`) e sua acurácia (`melhor_modelo.txt`).

Os resultados são salvos em `results/leave-one-out/`. Essa validação é mais exaustiva e sensível a overfitting, ideal quando se busca precisão máxima com poucos dados.

In [ ]:
# Lê o arquivo CSV com os dados
df = pd.read_csv(arquivo_treino_validacao)

# Cria o diretório de saída para resultados se não existir
output_dir = Path("results/leave-one-out")
output_dir.mkdir(parents=True, exist_ok=True)

# Limpa arquivos e subpastas antigas do diretório de resultados
for item in output_dir.iterdir():
    if item.is_file():
        item.unlink()  # Remove arquivos
    elif item.is_dir():
        shutil.rmtree(item)  # Remove diretórios

# Define estilo de visualização para gráficos
sns.set_style("whitegrid")

# Define o tamanho padrão dos gráficos
plt.rcParams['figure.figsize'] = (10, 6)

# Define a precisão de exibição de números no pandas
pd.set_option('display.precision', 3)

# Define os modelos que serão avaliados
algorithms = {
    "Decision Tree": DecisionTreeClassifier(max_depth=3, random_state=42),
    "Big Tree": DecisionTreeClassifier(random_state=42),
    "kNN": KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    "Naive Bayes": GaussianNB(),
    "SVM Linear": LinearSVC(C=0.1, max_iter=10000, random_state=42),
    "SVM RBF": SVC(gamma='scale', C=1, probability=False, cache_size=500, random_state=42)
}

# Função que agrupa listas de acurácias em blocos (para visualização mais clara do Boxplot)
def agrupa_media(accs, step=100):
    return [np.mean(accs[i:i+step]) for i in range(0, len(accs), step)]

# Avalia o desempenho de um modelo usando Leave-One-Out
# Retorna lista de acurácias, rótulos reais e previstos

def evaluate_model_fast(df, model_name):
    X = df.drop(columns=["gender"]).values  # Atributos
    y = df["gender"].values  # Classe alvo
    model = clone(algorithms[model_name])  # Clona o modelo da lista
    scaler = StandardScaler()  # Normalizador padrão
    accuracies = []  # Lista para armazenar acurácias
    y_true, y_pred = [], []  # Rótulos reais e previstos

    # Para cada divisão leave-one-out
    for train_idx, test_idx in LeaveOneOut().split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Normaliza com base no treino
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Treina o modelo
        model.fit(X_train, y_train)

        # Faz a predição
        pred = model.predict(X_test)

        # Armazena resultados
        accuracies.append(accuracy_score(y_test, pred))
        y_true.append(y_test[0])
        y_pred.append(pred[0])

    return np.array(accuracies), model_name, y_true, y_pred

# Gera gráfico boxplot a partir dos resultados

def plot_boxplot(result: dict, boxplot_file, y_label_text="Score", titulo="Boxplot de Resultados"):
    plt.style.use('default')  # Estilo padrão
    plt.figure(figsize=(10, 6))  # Tamanho do gráfico
    cleaned = [pd.Series(v).dropna() for v in result.values()]  # Remove NaNs
    plt.boxplot(cleaned, tick_labels=list(result.keys()))  # Gera o boxplot
    plt.title(titulo)
    plt.ylabel(y_label_text)
    plt.xticks(rotation=15)
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(boxplot_file, dpi=300)  # Salva como PNG
    plt.close()
    print(f"Boxplot salvo em: {boxplot_file}")

# Função principal para rodar a análise LOO completa

def run_analysis(df):
    resultados = {}  # Guarda todas as acurácias
    resultados_boxplot = {}  # Guarda resultados em blocos para gráfico
    melhor_acc = 0  # Armazena a melhor acurácia
    melhor_modelo_nome = ""  # Nome do melhor modelo
    relatorio_path = output_dir / "relatorio.txt"  # Caminho do relatório

    with open(relatorio_path, "w", encoding="utf-8") as f:
        f.write("Relatório Leave-One-Out:\n\n")

        for name in algorithms:
            # Avalia o modelo
            accs, _, y_true, y_pred = evaluate_model_fast(df, name)
            resultados[name] = accs
            resultados_boxplot[name] = agrupa_media(accs, step=100)

            # Calcula a média das acurácias
            media = np.mean(accs)
            if media > melhor_acc:
                melhor_acc = media
                melhor_modelo_nome = name

            # Gera relatório
            relatorio = classification_report(y_true, y_pred, target_names=["Women", "Men"], digits=4, zero_division=0)
            f.write(f"Modelo: {name}\n{relatorio}\n" + "-" * 60 + "\n")

            print(f"\nModelo: {name}")
            print(relatorio)
            print("-" * 60)

            # Gera matriz de confusão
            cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
            plt.figure(figsize=(6, 5))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                        xticklabels=["Women", "Men"], yticklabels=["Women", "Men"])
            plt.xlabel("Classe Predita")
            plt.ylabel("Classe Real")
            plt.title(f"Matriz de Confusão - {name}")
            plt.tight_layout()
            plt.savefig(output_dir / f"{name}_matriz_confusao.png", dpi=300)
            plt.close()

    # Gera boxplot final
    plot_boxplot(
        result=resultados_boxplot,
        boxplot_file=output_dir / "boxplot.png",
        y_label_text="Acurácia",
        titulo="Boxplot - Leave-One-Out (Agrupado por Blocos)"
    )

    # Gera gráfico de barras com acurácias médias
    df_plot = pd.DataFrame({
        "Modelo": list(resultados.keys()),
        "Acurácia": [np.mean(v) for v in resultados.values()]
    })

    plt.figure(figsize=(10, 6))
    sns.barplot(x="Modelo", y="Acurácia", data=df_plot)
    plt.ylim(0, 1.05)
    plt.title("Acurácia dos Modelos - Leave-One-Out")
    plt.ylabel("Acurácia")
    plt.xlabel("Modelos")
    plt.grid(True, axis="y", linestyle="--", alpha=0.6)
    plt.xticks(rotation=15)
    plt.tight_layout()
    plt.savefig(output_dir / "barplot.png", dpi=300)
    plt.close()
    print("Barplot salvo em:", output_dir / "barplot.png")

    # Prepara os dados para treino final
    X = df.drop(columns=["gender"]).values
    y = df["gender"].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Treina o melhor modelo com todos os dados
    modelo_final = clone(algorithms[melhor_modelo_nome])
    modelo_final.fit(X_scaled, y)

    # Salva informações do melhor modelo
    with open(output_dir / "melhor_modelo.txt", "w", encoding="utf-8") as f:
        f.write(f"Melhor modelo: {melhor_modelo_nome}\nAcurácia: {melhor_acc:.4f}")

    # Salva o scaler + modelo treinado juntos
    joblib.dump((scaler, modelo_final), output_dir / "melhor_modelo.pkl")

    print("\nAnálise concluída com sucesso!")
    print(f"\nMelhor modelo: {melhor_modelo_nome} (Acurácia: {melhor_acc:.4f})")
    print("Modelo final salvo em:", output_dir / "melhor_modelo.pkl")

# Executa a análise se o script for chamado diretamente
if __name__ == "__main__":
    run_analysis(df)

## -------------------- 11. Validação Cruzada com Grid Search --------------------

Esta etapa avalia o desempenho de múltiplos classificadores utilizando validação cruzada estratificada de 3 folds (3-fold Stratified Cross-Validation), com busca de hiperparâmetros (`GridSearchCV`) para alguns modelos.

**Modelos utilizados:**
- Decision Tree (com e sem limite de profundidade)
- k-Nearest Neighbors (k = 3, 5, 7)
- Naive Bayes
- Support Vector Machine (Linear e RBF com `SGDClassifier`)

**Técnicas aplicadas:**
1. **Normalização:** os dados são padronizados (`StandardScaler`).
2. **Nested Cross-Validation:** para cada modelo, aplica-se `GridSearchCV` internamente (se aplicável) e `cross_val_score` externamente.
3. **Avaliação Final:** após validação externa, o melhor modelo é treinado em todo o conjunto.
4. **Relatórios:** são geradas as matrizes de confusão, relatório de classificação, boxplots e gráficos de barras comparando acurácias médias.

O melhor modelo é salvo com o `scaler` em `results/cross-validation/melhor_modelo.pkl`.

In [ ]:
# Lê o banco de dados de treino/teste
df = pd.read_csv(arquivo_treino_validacao)

# Separa atributos (X) e rótulos (y)
X = df.drop(columns=["gender"])
y = df["gender"]

# 2. Cria e limpa diretório de resultados
output_dir = Path("results/cross-validation")
output_dir.mkdir(parents=True, exist_ok=True)

# Remove arquivos e subpastas antigas
for item in output_dir.iterdir():
    if item.is_file():
        item.unlink()
    elif item.is_dir():
        shutil.rmtree(item)

# 3. Padroniza os dados com média 0 e desvio padrão 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Define os modelos e seus hiperparâmetros (grid search)
modelos_com_grid = {
    "Decision Tree": (DecisionTreeClassifier(random_state=42), {
        'max_depth': [3, 5, 7, None],
        'criterion': ['gini', 'entropy']
    }),
    "Big Tree": (DecisionTreeClassifier(random_state=42), {}),
    "kNN": (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7]
    }),
    "Naive Bayes": (GaussianNB(), {}),
    "SVM Linear": (LinearSVC(random_state=42, dual=False, max_iter=10000), {
        'C': [0.1, 1, 10]
    }),
    "SVM RBF (Fast)": (SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, random_state=42), {
        'alpha': [0.0001, 0.001, 0.01]
    })
}

# 5. Define validações interna e externa (nested CV)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 6. Inicializa estruturas para armazenar resultados
resultados_val = {}
melhor_modelo = None
melhor_nome = ""
melhor_acc = 0
relatorio_path = output_dir / "relatorio.txt"

# 7. Avalia os modelos
with open(relatorio_path, "w", encoding="utf-8") as f:
    f.write("Relatório Cross-Validation (modelo treinado em todos os dados após validação externa):\n\n")

    # Itera sobre os modelos e grids
    for nome, (modelo, grid) in modelos_com_grid.items():
        print(f"\nModelo: {nome}")

        # Se houver grid, usa GridSearchCV com validação interna
        if grid:
            grid_search = GridSearchCV(modelo, grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)
        else:
            grid_search = modelo

        # Realiza validação cruzada externa
        scores = cross_val_score(grid_search, X=X_scaled, y=y, cv=outer_cv, n_jobs=-1)
        resultados_val[nome] = scores

        # Treina modelo final com todos os dados para gerar relatório
        if hasattr(grid_search, 'fit'):
            grid_search.fit(X_scaled, y)
            final_model = grid_search.best_estimator_ if hasattr(grid_search, 'best_estimator_') else grid_search
        else:
            final_model = modelo.fit(X_scaled, y)

        # Prediz rótulos e calcula acurácia
        y_pred = final_model.predict(X_scaled)
        acc = accuracy_score(y, y_pred)

        # Gera e escreve relatório de classificação
        relatorio = classification_report(y, y_pred, target_names=["Women", "Men"], digits=4)
        f.write(f"Modelo: {nome}\n")
        f.write(relatorio)
        f.write("\n" + "-" * 60 + "\n")
        print(relatorio)

        # Gera e salva matriz de confusão
        cm = confusion_matrix(y, y_pred, labels=[0, 1])
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["Women", "Men"], yticklabels=["Women", "Men"])
        plt.xlabel("Classe Predita")
        plt.ylabel("Classe Real")
        plt.title(f"Matriz de Confusão - {nome}")
        plt.tight_layout()
        plt.savefig(output_dir / f"{nome}_matriz_confusao.png", dpi=300)
        plt.close()

        # Atualiza melhor modelo, se necessário
        if acc > melhor_acc:
            melhor_acc = acc
            melhor_modelo = final_model
            melhor_nome = nome

# 8. Gera boxplot com acurácias da validação cruzada
def plot_boxplot(result, boxplot_file, y_label_text="Score", titulo="Boxplot"):
    plt.style.use('default')
    plt.figure(figsize=(10, 6))
    cleaned = [pd.Series(v).dropna() for v in result.values()]
    plt.boxplot(cleaned, tick_labels=list(result.keys()))
    plt.title(titulo)
    plt.ylabel(y_label_text)
    plt.xticks(rotation=15)
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(boxplot_file, dpi=300)
    plt.close()
    print(f"Boxplot salvo em: {boxplot_file}")

plot_boxplot(
    result=resultados_val,
    boxplot_file=output_dir / "boxplot.png",
    y_label_text="Acurácia",
    titulo="Boxplot - Cross-Validation"
)

# 9. Gera gráfico de barras com média e desvio padrão
df_stats = pd.DataFrame({
    "Modelo": list(resultados_val.keys()),
    "Acurácia Média": [np.mean(v) for v in resultados_val.values()],
    "Desvio Padrão": [np.std(v) for v in resultados_val.values()]
})

# Cria o gráfico de barras
plt.figure(figsize=(12, 7))
sns.barplot(x="Modelo", y="Acurácia Média", data=df_stats)
plt.grid(True, axis='y', linestyle='--', alpha=0.6)
plt.title("Acurácia Média por Modelo", pad=15, fontweight='bold')
plt.ylim(0, 1.05)
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(output_dir / "barplot.png", dpi=300)
plt.close()

# 10. Salva o melhor modelo e o scaler em disco
joblib.dump((scaler, melhor_modelo), output_dir / "melhor_modelo.pkl")

# Salva nome e acurácia do melhor modelo
with open(output_dir / "melhor_modelo.txt", "w", encoding="utf-8") as f:
    f.write(f"Melhor modelo: {melhor_nome}\nAcurácia: {melhor_acc:.4f}")

# Mensagens finais
print("\nAnálise concluída com sucesso!")
print(f"Melhor modelo: {melhor_nome} (Acurácia: {melhor_acc:.4f})")
print("Resultados salvos em:", output_dir)


## -------------------- 12. Comparação Visual dos Classificadores em Dados Sintéticos --------------------

Esta etapa tem como objetivo **visualizar o comportamento dos classificadores** utilizados no projeto
em **dados sintéticos bidimensionais**.

São usados três conjuntos de dados artificiais:
- Moons (semiluas com ruído)
- Circles (círculos concêntricos)
- Dados gaussianos simulados com 2 atributos informativos

Seis classificadores são treinados em cada conjunto e as **fronteiras de decisão** são desenhadas, 
permitindo comparar:
- Como cada modelo separa regiões de decisão
- Robustez frente a diferentes distribuições
- Comportamento visual (overfitting vs generalização)

O resultado é um gráfico com 3 linhas (uma para cada dataset) e 7 colunas (dados de entrada + 6 classificadores).

Este gráfico não utiliza os dados reais do projeto (dados CSI). Ele tem caráter ilustrativo, gerando dados sintéticos bidimensionais para demonstrar visualmente como os classificadores se comportam em diferentes cenários. Serve como apoio didático e não interfere diretamente nos resultados do experimento com os dados reais.

O arquivo gerado é salvo como `results/classifier_comparison.png`.

In [ ]:
# Lista com nomes dos classificadores utilizados
names = [
    "kNN", "Decision Tree", "Big Tree", "Naive Bayes", "SVM Linear", "SVM RBF"
]

# Lista com instâncias dos classificadores configurados
classifiers = [
    KNeighborsClassifier(3),                      # kNN com k=3
    DecisionTreeClassifier(max_depth=5),          # Árvore com profundidade limitada
    DecisionTreeClassifier(),                     # Árvore sem limitação de profundidade
    GaussianNB(),                                 # Classificador Naive Bayes
    SVC(kernel="linear", C=0.025, probability=True),  # SVM com kernel linear
    SVC(gamma=2, C=1, probability=True),              # SVM com kernel RBF
]

# Conjuntos de dados sintéticos para visualização das fronteiras de decisão
datasets = [
    make_moons(noise=0.3, random_state=0),        # Dados em formato de semiluas
    make_circles(noise=0.2, factor=0.5, random_state=1),  # Dados em círculos concêntricos
    make_classification(                          # Dados gaussianos com 2 atributos informativos
        n_features=2, n_redundant=0, n_informative=2,
        random_state=42, n_clusters_per_class=1
    )
]

# Ignora avisos de convergência, especialmente úteis para modelos como MLP (não usados aqui, mas por segurança)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

    # Cria figura com layout grande para todas as subplots
    figure = plt.figure(figsize=(20, 6))

    # Índice da subplot
    i = 1

    # Itera sobre os datasets sintéticos
    for ds in datasets:
        # Separa atributos e rótulos
        X, y = ds

        # Normaliza os atributos
        X = StandardScaler().fit_transform(X)

        # Separa dados de treino e teste (60/40)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

        # Primeira coluna: dados de entrada
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        ax.set_title("Input data")
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
        i += 1

        # Colunas seguintes: classificadores aplicados
        for name, clf in zip(names, classifiers):
            ax = plt.subplot(len(datasets), len(classifiers) + 1, i)

            # Treina o classificador
            clf.fit(X_train, y_train)

            # Avalia acurácia no conjunto de teste
            score = clf.score(X_test, y_test)

            # Gera a visualização da fronteira de decisão
            display = DecisionBoundaryDisplay.from_estimator(
                clf, X, response_method="predict", cmap=plt.cm.coolwarm, alpha=0.8, ax=ax
            )

            # Sobrepõe os pontos de teste
            ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=plt.cm.coolwarm, edgecolors='k')

            # Título do gráfico com nome e acurácia
            ax.set_title(f"{name}\nAccuracy: {score:.2f}")
            i += 1

    # Ajusta o layout geral da figura
    plt.tight_layout()

    # Salva o gráfico final em alta resolução
    plt.savefig("results/classifier_comparison.png", dpi=300)

    # Fecha a figura
    plt.close()

    # Mensagem informando o local do arquivo salvo
    print("Gráfico de Comparação Salvo em: results/classifier_comparison.png")

## -------------------- 13. Análise com Árvore / Visualização de Modelos --------------------

Este bloco exporta e visualiza os modelos finais que foram salvos durante as etapas de validação (Hold-Out, Cross-Validation e Leave-One-Out), mas **somente se o modelo salvo for uma `DecisionTreeClassifier`**.

**O que ele faz:**

1. Lê novamente os dados originais e prepara `X` e `y` com `convert_gender_to_numeric`.
2. Verifica os caminhos dos modelos salvos em:
   - `results/hold-out/melhor_modelo.pkl`
   - `results/cross-validation/melhor_modelo.pkl`
   - `results/leave-one-out/melhor_modelo.pkl`
3. Para cada modelo:
   - Carrega o arquivo `.pkl`.
   - Verifica se é uma árvore de decisão.
   - Se for, gera um `.dot` com `export_graphviz` e exporta o gráfico `.png` com `graphviz`.

Os arquivos `.png` gerados são salvos em `results/arvores/arvore_<tecnica>_<modelo>.png`.

Isso permite inspecionar visualmente a lógica usada pelos classificadores baseados em árvore.

In [ ]:
# Carrega o dataset de treino/validação a partir do arquivo CSV
df = pd.read_csv(arquivo_treino_validacao)

# Converte a coluna 'gender' para valores numéricos (f → 0, m → 1)
df_full = convert_gender_to_numeric(df.copy())

# Define o diretório onde as árvores serão salvas
output_dir = Path("results/arvores")

# Cria o diretório se ele ainda não existir
output_dir.mkdir(parents=True, exist_ok=True)

# Remove arquivos e subpastas antigas do diretório
for item in output_dir.iterdir():
    if item.is_file():
        item.unlink()  # Remove arquivo individual
    elif item.is_dir():
        shutil.rmtree(item)  # Remove subdiretório inteiro

# Separa os atributos (X) e a variável alvo (y)
X = df_full.drop("gender", axis=1)
y = df_full["gender"]

# Define os caminhos dos melhores modelos salvos de cada técnica
avaliacoes = {
    "hold-out": Path("results/hold-out/melhor_modelo.pkl"),
    "cross-validation": Path("results/cross-validation/melhor_modelo.pkl"),
    "leave-one-out": Path("results/leave-one-out/melhor_modelo.pkl")
}

# Percorre cada técnica e seu respectivo caminho de modelo salvo
for tecnica, caminho_modelo in avaliacoes.items():
    
    # Verifica se o arquivo do modelo existe
    if not caminho_modelo.exists():
        print(f"Modelo não encontrado: {caminho_modelo}")
        continue

    # Carrega o modelo salvo com joblib
    print(f"Carregando modelo salvo de: {caminho_modelo}")
    modelo_geral = joblib.load(caminho_modelo)

    # Se o modelo foi salvo como tupla (scaler, modelo), extrai só o modelo
    if isinstance(modelo_geral, tuple):
        _, modelo_geral = modelo_geral

    # Ignora se o modelo não for uma árvore de decisão
    if not isinstance(modelo_geral, DecisionTreeClassifier):
        print(f"Ignorado: {tecnica} não é DecisionTreeClassifier.\n")
        continue

    # Obtém o nome da classe do modelo (ex: DecisionTreeClassifier)
    nome_modelo = modelo_geral.__class__.__name__

    # Constrói um nome seguro para o arquivo, substituindo caracteres especiais
    nome_seguro = re.sub(r"[^\w\-]", "_", nome_modelo)

    # Define caminho do arquivo .dot (estrutura da árvore)
    dot_path = Path(f"results/arvores/arvore_{tecnica}_{nome_seguro}.dot")

    # Define caminho da imagem .png gerada a partir da árvore
    png_path = Path(f"results/arvores/arvore_{tecnica}_{nome_seguro}.png")

    # Exporta a árvore de decisão para o formato .dot
    export_graphviz(
        modelo_geral,
        out_file=str(dot_path),
        feature_names=X.columns,
        class_names=["Women", "Men"],
        rounded=True,
        filled=True
    )

    # Lê o arquivo .dot e renderiza como imagem .png
    graph = Source.from_file(str(dot_path))
    graph.render(str(png_path.with_suffix("")), format="png", cleanup=True)

    # Informa no terminal que a árvore foi salva
    print(f"Árvore salva como: {png_path}")

## -------------------- 14. Geração das Curvas ROC --------------------

Esta etapa visualiza a **performance dos classificadores** na distinção entre as classes “Men” e “Women” com base nas curvas ROC (Receiver Operating Characteristic), que comparam a taxa de verdadeiros positivos (TPR) com a de falsos positivos (FPR). Também é exibida a AUC (Área sob a Curva), métrica que representa a capacidade de separação do modelo.

**Etapas executadas:**

1. **Leitura e preparação do conjunto real de teste.**
2. **Padronização dos dados com `StandardScaler`.**
3. **Geração das curvas ROC para 3 técnicas:**
   - **Hold-Out:** treino/validação com split 80/20
   - **Cross-Validation (3-fold):** agregação das predições ao longo das dobras
   - **Leave-One-Out:** uma predição por rodada
4. **Para cada técnica e classificador:**
   - Aplica o modelo sobre os dados padronizados
   - Usa `predict_proba` ou `decision_function` para obter scores
   - Salva um gráfico `PNG` por técnica em `results/curvas_roc/roc_<tecnica>.png`

Isso permite comparar visualmente o desempenho dos modelos, inclusive com diferentes limiares de decisão.


In [ ]:
# Lê o dataset real de teste com rótulo "gender"
df = pd.read_csv(arquivo_teste_real)

# Cria diretório onde os gráficos de curva ROC serão salvos
roc_dir = Path("results/curvas_roc")
roc_dir.mkdir(parents=True, exist_ok=True)

# Limpa todos os arquivos e pastas dentro do diretório de resultados
for item in roc_dir.iterdir():
    if item.is_file():
        item.unlink()
    elif item.is_dir():
        shutil.rmtree(item)

# Função que gera e salva curvas ROC para um conjunto de classificadores
def plot_save_roc_curves(y_true_dict, y_score_dict, technique_name):
    plt.figure(figsize=(10,7))
    sns.set_style("whitegrid")
    cores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

    for idx, model_name in enumerate(y_true_dict.keys()):
        y_true = y_true_dict[model_name]
        y_score = y_score_dict[model_name]

        y_true = np.array(y_true)
        y_score = np.array(y_score)

        # Garante que existem pelo menos duas classes
        if len(np.unique(y_true)) < 2 or len(np.unique(y_score)) < 2:
            continue

        # Gera pontos da curva ROC e calcula AUC
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        # Plota a curva ROC com rótulo do modelo e AUC
        plt.plot(fpr, tpr, label=f"{model_name} (AUC={roc_auc:.3f})", lw=2, color=cores[idx % len(cores)])

    # Linha de referência aleatória
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Aleatório (AUC = 0.5)')

    # Define eixos e título
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("Falso Positivo (FPR)", fontsize=12)
    plt.ylabel("Verdadeiro Positivo (TPR)", fontsize=12)
    plt.title(f"Curva ROC - {technique_name}", fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.4)
    plt.tight_layout()

    # Salva gráfico
    fname = roc_dir / f"roc_{technique_name.replace(' ', '_').lower()}.png"
    plt.savefig(fname, dpi=300)
    plt.close()
    print(f"Curva roc de {technique_name} salva em {fname}")

# ----------------------------------------------------------------
# Etapa comum: separa atributos e rótulo, normaliza os dados
X = df.drop(columns=["gender"]).values
y = df["gender"].values
scaler = StandardScaler()
# ----------------------------------------------------------------

# GERAÇÃO DA CURVA ROC PARA HOLD-OUT
print("Gerando curva roc de Hold-Out")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_true_dict_ho = {}
y_score_dict_ho = {}

# Para cada modelo, treina e gera scores de probabilidade ou decisão
for name, modelo in algorithms.items():
    clf = clone(modelo)
    clf.fit(X_train, y_train)
    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)[:, 1]
    elif hasattr(clf, "decision_function"):
        y_score = clf.decision_function(X_test)
    else:
        y_score = clf.predict(X_test)
    y_true_dict_ho[name] = y_test
    y_score_dict_ho[name] = y_score

# Gera a curva ROC do Hold-Out
plot_save_roc_curves(y_true_dict_ho, y_score_dict_ho, "Hold-Out")

# GERAÇÃO DA CURVA ROC PARA CROSS-VALIDATION
print("Gerando curva roc de Cross-Validation")
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
X_scaled = scaler.fit_transform(X)
y_true_dict_cv = {}
y_score_dict_cv = {}

for name, modelo in algorithms.items():
    y_true_cv = []
    y_score_cv = []
    for train_idx, test_idx in skf.split(X_scaled, y):
        clf = clone(modelo)
        clf.fit(X_scaled[train_idx], y[train_idx])
        if hasattr(clf, "predict_proba"):
            y_score = clf.predict_proba(X_scaled[test_idx])[:, 1]
        elif hasattr(clf, "decision_function"):
            y_score = clf.decision_function(X_scaled[test_idx])
        else:
            y_score = clf.predict(X_scaled[test_idx])
        y_true_cv.extend(y[test_idx])
        y_score_cv.extend(y_score)
    y_true_dict_cv[name] = np.array(y_true_cv)
    y_score_dict_cv[name] = np.array(y_score_cv)

# Gera a curva ROC do Cross-Validation
plot_save_roc_curves(y_true_dict_cv, y_score_dict_cv, "Cross-Validation")

# GERAÇÃO DA CURVA ROC PARA LEAVE-ONE-OUT
print("Gerando curva roc de Leave-One-Out")
loo = LeaveOneOut()
X_scaled = scaler.fit_transform(X)
y_true_dict_loo = {}
y_score_dict_loo = {}

for name, modelo in algorithms.items():
    y_true_loo = []
    y_score_loo = []
    for train_idx, test_idx in loo.split(X_scaled):
        clf = clone(modelo)
        clf.fit(X_scaled[train_idx], y[train_idx])
        if hasattr(clf, "predict_proba"):
            y_score = clf.predict_proba(X_scaled[test_idx])[:, 1]
        elif hasattr(clf, "decision_function"):
            y_score = clf.decision_function(X_scaled[test_idx])
        else:
            y_score = clf.predict(X_scaled[test_idx])
        y_true_loo.append(y[test_idx][0])
        y_score_loo.append(y_score[0])
    y_true_dict_loo[name] = np.array(y_true_loo)
    y_score_dict_loo[name] = np.array(y_score_loo)

# Gera a curva ROC do Leave-One-Out
plot_save_roc_curves(y_true_dict_loo, y_score_dict_loo, "Leave-One-Out")

## -------------------- 15. Avaliação Final - Teste Real --------------------

Nesta etapa, os modelos finalistas de cada técnica (Hold-Out, Cross-Validation e Leave-One-Out) são aplicados ao conjunto real de teste (`db_800.csv`) para avaliar sua capacidade de generalização.

Além do relatório de classificação salvo em `.txt`, é gerada uma imagem da matriz de confusão para cada modelo avaliado, permitindo análise visual da performance.

**O que o código realiza:**
- Suprime `warnings` de conversão e usuários para facilitar a leitura dos logs
- Lê os dados reais de teste
- Aplica o modelo salvo para cada técnica, com ou sem `StandardScaler`
- Detecta o nome do classificador, tratando `DecisionTreeClassifier(max_depth=None)` como "Big Tree"
- Gera:
  - Relatórios de classificação salvos em `results/teste_real/`
  - Matrizes de confusão salvas como imagens `PNG` no mesmo diretório
  - Impressão dos relatórios no terminal com título da técnica e modelo

Esse processo finaliza a comparação objetiva dos modelos com dados nunca vistos.

In [ ]:
# Suprime warnings indesejados durante execução dos classificadores
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DataConversionWarning)

# Lê o conjunto real de teste
df_teste = pd.read_csv(arquivo_teste_real)

# Separa os atributos (X_test) e os rótulos (y_test)
X_test = df_teste.drop("gender", axis=1)  # Remove coluna de classe
y_test = df_teste["gender"]  # Define variável alvo

# Define os caminhos dos modelos salvos (1 por técnica)
tecnicas = {
    "hold-out": "results/hold-out/melhor_modelo.pkl",    
    "leave-one-out": "results/leave-one-out/melhor_modelo.pkl",
    "cross-validation": "results/cross-validation/melhor_modelo.pkl"
}

# Cria diretório onde os relatórios e gráficos serão salvos
dir_saida = Path("results/teste_real")
dir_saida.mkdir(parents=True, exist_ok=True)

# Executa a avaliação para cada técnica
for tecnica, caminho_modelo in tecnicas.items():
    try:
        # Carrega o modelo (pode ser tupla: (scaler, modelo))
        modelo_carregado = joblib.load(caminho_modelo)

        # Se for tupla, aplica o scaler ao X_test
        if isinstance(modelo_carregado, tuple):
            scaler, modelo = modelo_carregado
            X_test_scaled = scaler.transform(X_test.values)
        else:
            modelo = modelo_carregado
            X_test_scaled = X_test.values  # Dados crus

        # Detecta nome do modelo, tratando árvore sem max_depth como "Big Tree"
        nome_base = type(modelo).__name__
        params = modelo.get_params()
        if nome_base == "DecisionTreeClassifier" and params.get("max_depth", 1) is None:
            nome_modelo_humano = "Big Tree"
        else:
            nome_modelo_humano = nome_base

        # Realiza predição sobre o teste real
        y_pred = modelo.predict(X_test_scaled)

        # Gera métricas de avaliação
        acc = accuracy_score(y_test, y_pred)  # Acurácia
        relatorio = classification_report(
            y_test, y_pred, target_names=["Women", "Men"], digits=4, zero_division=0
        )
        matriz = confusion_matrix(y_test, y_pred, labels=[0, 1])  # Matriz confusão

        # Formata título para relatório
        titulo = f"Técnica: {tecnica.replace('-', ' ').title()} | Modelo: {nome_modelo_humano}"

        # Salva o relatório de classificação em .txt
        caminho_relatorio = dir_saida / f"relatorio_{tecnica}_{nome_modelo_humano}.txt"
        with open(caminho_relatorio, "w", encoding="utf-8") as f:
            f.write(f"{titulo}\n{relatorio}")
        print(f"\n\n{titulo}\n{relatorio}")
        print(f"Relatório do melhor modelo de {tecnica} salvo em {caminho_relatorio}")        

        # Gera e salva a imagem da matriz de confusão
        plt.figure(figsize=(6, 5))
        sns.heatmap(matriz, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["Women", "Men"], yticklabels=["Women", "Men"])
        plt.xlabel("Classe Predita")
        plt.ylabel("Classe Real")
        plt.title(f"{tecnica.replace('-', ' ').title()} - {nome_modelo_humano} - TESTE REAL")
        plt.tight_layout()
        caminho_img = dir_saida / f"matriz_confusao_{tecnica}_{nome_modelo_humano}.png"
        plt.savefig(caminho_img, dpi=300)
        plt.close()
        print(f"Matriz de confusão do melhor modelo de {tecnica} salva em {caminho_img}")

    # Caso ocorra erro ao processar alguma técnica (ex: arquivo faltando)
    except Exception as e:
        print(f"\nErro com técnica '{tecnica}': {e}")

## -------------------- Fim do código --------------------

In [ ]:
fim = time()
duracao = strftime('%H:%M:%S', gmtime(fim - inicio))
print(f"\nTempo execução (HH:mm:ss): {duracao}")